### Rastreo de noticias en Google News sobre temas de la Ciudad de México que competan o sean de interés del Instituto de Planeación de la Ciudad de México

El script extrae las notas de Google News de los temas de agua, movilidad, planeación, salud y seguridad de la Ciudad de México. El script extrae los títulos, url y fecha de las notas.
Además se ordena y extrae en el formato de entrega diaria para el Instituto de Planeación.

In [12]:
import requests
import pandas as pd
import re
from bs4 import BeautifulSoup
from docx import Document
from docx.shared import RGBColor
from docx.oxml import OxmlElement
from docx.oxml.ns import qn
from docx.shared import Pt
import locale
import os
from rapidfuzz import fuzz
import unicodedata

Extraer notas a partir de los temas relevantes y palabras clave para localizarlos en Ciudad de México

In [13]:
# Temas para la búsqueda
temas = ["clara%20brugada",
    "planeación", "infraestructura", "agua", "pgd", "pgot", "metropolitano"
    "consulta%20pública", "consulta%20indígena", "vivienda",
    "ordenamiento%20territorial", "movilidad", 
    "salud", "gestión%20de%20riesgos", "seguridad%20ciudadana", 
    "participación%20ciudadana", "medio%20ambiente", "cultura", 
    "asentamientos%20irregulares", "ipdp", "congreso%20de%20la%20ciudad%20de%20méxico",
    "transporte%20público", "espacio%20público",
    "patricia%20ramírez%20kuri"  
]

In [14]:
def limpiar_tema(t):
    t = t.replace('%20', ' ')
    palabras = t.split()
    excepciones = ['de', 'la', 'del', 'y', 'en', 'a']  # palabras que no se capitalizan
    palabras_limpias = []

    for i, palabra in enumerate(palabras):
        if i == 0 or palabra.lower() not in excepciones:
            palabras_limpias.append(palabra.capitalize())
        else:
            palabras_limpias.append(palabra.lower())
    
    return ' '.join(palabras_limpias)

In [15]:
#Palabras clave para filtrar solo noticias de la Ciudad de México
palabras_clave_cdmx = [
    "Ciudad de México", "CDMX", "Iztapalapa", "Coyoacán", "Cuauhtémoc", "Benito Juárez", 
    "Miguel Hidalgo", "Xochimilco", "Tlalpan", "Gustavo A. Madero", "Venustiano Carranza",
    "Azcapotzalco", "Tláhuac", "Álvaro Obregón", "Milpa Alta", "Magdalena Contreras", "Brugada"
]

In [ ]:
#Lista de medios relevantes
medios = ["CDMX","Imagen Radio 90.5","Aristegui Noticias",
          "88.9 Noticias","W Radio México", "La Octava",
          "Telediario México","capital21.cdmx.gob.mx",
          "CDMX Magacín",
    'infobae', 'La Prensa', 'Debate', 'La Crónica de Hoy', 'El Universal',
    'Reforma', 'MVS Noticias', 'SinEmbargo MX', 'Cuarto Poder', 'Reporte Indigo',
    'Eje Central', 'Gobierno de la Ciudad de México', 'La Silla Rota', 'MSN', 'Animal Político', 'La Razón de México',
    'Excélsior', 'Radio Fórmula', 'N+', 'La Izquierda Diario', 'Milenio',
    'Secretaría de Seguridad Ciudadana de la CDMX', 'Infobae México', 
    'TV Azteca', 'Quadratín México', 'ContraRéplica',
    'La Jornada', 'Uno TV Noticias', 'Secretaría de Protección Civil CDMX',
    'Ovaciones', 'ADN 40', 'El Heraldo de México',
    'IECM', 'El Economista', 'RTVE', 'EL PAÍS', 'Radio Fórmula'
]

#Convertir la lista de medios en una expresión regular
medios_regex = '|'.join(map(re.escape, medios))

In [17]:
#Función para extraer notas
def get_notas(tema, periodo):
    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/134.0.0.0 Safari/537.36'}
    
    url = f'https://news.google.com/search?q={tema}%20ciudad%20de%20mexico%20when%3A{periodo}d&hl=es-419&gl=MX&ceid=MX%3Aes-419'
    response = requests.get(url, headers=headers)
    sopa = BeautifulSoup(response.text, 'html.parser')
    resultados = sopa.find_all('article')
    
    #Listas para almacenar datos
    titulos, urls, fechas, medios_lista = [], [], [], []

    for resultado in resultados:
        #Extraer título
        titulo_elem = resultado.find('a', class_='JtKRv')
        titulo = titulo_elem.text.strip() if titulo_elem else None

        #Extraer URL
        url_elem = resultado.find('a')
        url = 'https://news.google.com' + url_elem.get('href')[1:] if url_elem and url_elem.get('href') else None

        #Extraer fecha
        fecha_elem = resultado.find('time')
        fecha = fecha_elem['datetime'] if fecha_elem else None

        #Extraer medio
        medio_elem = resultado.find('div', class_='vr1PYe')
        medio = medio_elem.text.strip() if medio_elem else None

        #Guardar en listas
        titulos.append(titulo)
        urls.append(url)
        fechas.append(fecha)
        medios_lista.append(medio)

    #Crear DataFrame
    df = pd.DataFrame({
        'titulo': titulos,
        'url': urls,
        'fecha_nota': fechas,
        'medio': medios_lista,
        'fecha_consulta': pd.to_datetime('today').strftime('%Y-%m-%d'),
        'tema': tema
    })

    return df

In [18]:
def filtrar_notas(df, umbral=85, medios_preferidos=None):
    if medios_preferidos is None:
        medios_preferidos = []

    df_filtrado = pd.DataFrame(columns=df.columns)
    titulos_grupos = []

    for idx, row in df.iterrows():
        titulo_actual = row['titulo']
        grupo_encontrado = False

        # Buscar si el título se parece a alguno ya registrado
        for grupo in titulos_grupos:
            if any(fuzz.token_set_ratio(titulo_actual, t) >= umbral for t in grupo['titulos']):
                grupo['candidatos'].append(row)
                grupo['titulos'].append(titulo_actual)
                grupo_encontrado = True
                break

        # Si no se parece a ninguno, se crea un nuevo grupo
        if not grupo_encontrado:
            titulos_grupos.append({
                'titulos': [titulo_actual],
                'candidatos': [row]
            })

    # Para cada grupo, elegir el mejor candidato
    for grupo in titulos_grupos:
        candidatos = pd.DataFrame(grupo['candidatos'])
        preferidos = candidatos[candidatos['medio'].isin(medios_preferidos)]

        if not preferidos.empty:
            df_filtrado = pd.concat([df_filtrado, preferidos.iloc[[0]]], ignore_index=True)
        else:
            df_filtrado = pd.concat([df_filtrado, candidatos.iloc[[0]]], ignore_index=True)
#Eliminar notas de Hoy No Circula
    df_filtrado = df_filtrado[~df_filtrado['titulo'].str.contains("Hoy No Circula", case=False, na=False)]
    df_filtrado = df_filtrado[~df_filtrado['titulo'].str.contains("marchas hoy", case=False, na=False)]
    df_filtrado = df_filtrado[~df_filtrado['titulo'].str.contains("Marchas HOY", case=False, na=False)]
    df_filtrado = df_filtrado[~df_filtrado['titulo'].str.contains("marchas y bloqueos", case=False, na=False)]


    return df_filtrado.reset_index(drop=True)

In [19]:
periodo =  1 #Definir el periodo de busqueda por días 

#Extraer notas de todos los temas en un solo dataframe
consolidado_notas = pd.concat([get_notas(tema, periodo) for tema in temas], ignore_index=True)

#Filtrar notas de los medios definidos
consolidado_filtrado = consolidado_notas[consolidado_notas['medio'].fillna('').str.contains(medios_regex, regex=True)]

#Filtrar por palabras clave de CDMX
consolidado_final = consolidado_filtrado[consolidado_filtrado['titulo'].fillna('').str.contains('|'.join(palabras_clave_cdmx), regex=True, case=False)]


In [20]:
#Mostrar resultados
print(f'Total de notas encontradas: {len(consolidado_notas)}')
print(f'Notas filtradas por medios: {len(consolidado_filtrado)}')
print(f'Notas finales filtrando por palabras clave: {len(consolidado_final)}')

Total de notas encontradas: 678
Notas filtradas por medios: 253
Notas finales filtrando por palabras clave: 177


In [21]:
# Para hacer el ranking por fecha (más reciente primero)
consolidado_final['fecha_nota'] = pd.to_datetime(consolidado_final['fecha_nota'], errors='coerce')

C:\Users\claud\AppData\Local\Temp\ipykernel_28564\868838740.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  consolidado_final['fecha_nota'] = pd.to_datetime(consolidado_final['fecha_nota'], errors='coerce')


In [22]:
# Asegurarnos de que la columna de fecha no tenga zona horaria
consolidado_final['fecha_nota'] = consolidado_final['fecha_nota'].dt.tz_localize(None)

# Ahora podemos guardar en Excel sin problemas
consolidado_final.to_excel("noticias_cdmx.xlsx", index=False)

C:\Users\claud\AppData\Local\Temp\ipykernel_28564\1528448663.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  consolidado_final['fecha_nota'] = consolidado_final['fecha_nota'].dt.tz_localize(None)


Almacenar las notas en el formato de entrega

In [23]:
# Función para agregar hipervínculos con fuente Arial
def add_hyperlink(paragraph, text, url):
    part = paragraph._parent.part
    r_id = part.relate_to(url, "http://schemas.openxmlformats.org/officeDocument/2006/relationships/hyperlink", is_external=True)

    hyperlink = OxmlElement("w:hyperlink")
    hyperlink.set(qn("r:id"), r_id)

    run = OxmlElement("w:r")
    rPr = OxmlElement("w:rPr")

    # Fuente Arial
    rFonts = OxmlElement("w:rFonts")
    rFonts.set(qn("w:ascii"), "Arial")
    rFonts.set(qn("w:hAnsi"), "Arial")
    rFonts.set(qn("w:eastAsia"), "Arial")
    rPr.append(rFonts)

    # Color azul
    color = OxmlElement("w:color")
    color.set(qn("w:val"), "0000FF")
    rPr.append(color)

    # Subrayado
    underline = OxmlElement("w:u")
    underline.set(qn("w:val"), "single")
    rPr.append(underline)

    run.append(rPr)

    text_element = OxmlElement("w:t")
    text_element.text = text
    run.append(text_element)

    hyperlink.append(run)
    paragraph._element.append(hyperlink)

In [24]:
# Función de respaldo si locale falla
def fecha_en_espanol(fecha):
    meses = {
        'January': 'enero', 'February': 'febrero', 'March': 'marzo', 'April': 'abril',
        'May': 'mayo', 'June': 'junio', 'July': 'julio', 'August': 'agosto',
        'September': 'septiembre', 'October': 'octubre', 'November': 'noviembre', 'December': 'diciembre'
    }
    fecha_str = fecha.strftime('%d de %B de %Y')
    for en, es in meses.items():
        fecha_str = fecha_str.replace(en, es)
    return fecha_str

In [25]:
def normalizar_texto(texto):
    if not isinstance(texto, str):
        return ""
    texto = texto.lower()
    texto = unicodedata.normalize('NFD', texto).encode('ascii', 'ignore').decode('utf-8')  # quitar acentos
    texto = re.sub(r'\s+', ' ', texto).strip()  # eliminar espacios múltiples
    return texto

def filtrar_notas(df, umbral=85, medios_preferidos=None):
    if medios_preferidos is None:
        medios_preferidos = []

    df = df.copy()
    df['titulo_normalizado'] = df['titulo'].apply(normalizar_texto)

    df_filtrado = pd.DataFrame(columns=df.columns)
    titulos_grupos = []

    for idx, row in df.iterrows():
        titulo_actual = row['titulo_normalizado']
        grupo_encontrado = False

        # Buscar si el título se parece a alguno ya registrado
        for grupo in titulos_grupos:
            if any(
                max(
                    fuzz.token_sort_ratio(titulo_actual, t),
                    fuzz.partial_ratio(titulo_actual, t)
                ) >= umbral
                for t in grupo['titulos']
            ):
                grupo['candidatos'].append(row)
                grupo['titulos'].append(titulo_actual)
                grupo_encontrado = True
                break

        # Si no se parece a ninguno, se crea un nuevo grupo
        if not grupo_encontrado:
            titulos_grupos.append({
                'titulos': [titulo_actual],
                'candidatos': [row]
            })

    # Para cada grupo, elegir el mejor candidato
    for grupo in titulos_grupos:
        candidatos = pd.DataFrame(grupo['candidatos'])
        preferidos = candidatos[candidatos['medio'].isin(medios_preferidos)]

        if not preferidos.empty:
            df_filtrado = pd.concat([df_filtrado, preferidos.iloc[[0]]], ignore_index=True)
        else:
            df_filtrado = pd.concat([df_filtrado, candidatos.iloc[[0]]], ignore_index=True)

    # Eliminar columna auxiliar
    df_filtrado.drop(columns='titulo_normalizado', errors='ignore', inplace=True)
    df_filtrado = df_filtrado[~df_filtrado['titulo'].str.contains("Hoy No Circula", case=False, na=False)]
    df_filtrado = df_filtrado[~df_filtrado['titulo'].str.contains("marchas hoy", case=False, na=False)]
    df_filtrado = df_filtrado[~df_filtrado['titulo'].str.contains("Marchas HOY", case=False, na=False)]
    df_filtrado = df_filtrado[~df_filtrado['titulo'].str.contains("Marchas y movilizaciones", case=False, na=False)]

    return df_filtrado.reset_index(drop=True)

In [54]:
#Cargar la tabla de excel
df = pd.read_excel("noticias_cdmx.xlsx")
df['medio'] = df['medio'].fillna('Fuente desconocida')
df['url'] = df['url'].fillna('#')
df['fecha_nota'] = pd.to_datetime(df['fecha_nota'], errors='coerce').dt.strftime('%Y-%m-%d')

def limpiar_tema(t):
    return str(t).strip().capitalize()

df['tema'] = df['tema'].apply(limpiar_tema)
df['tema'] = df['tema'].str.replace('%20', ' ', regex=False)

# Agrupar y deduplicar temas
df['tema_normalizado'] = df['tema'].str.lower().str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8')
df_grouped = df.groupby(['fecha_nota', 'medio', 'titulo', 'url']).agg({
    'tema_normalizado': lambda x: ', '.join(sorted(set(t.capitalize() for t in x)))
}).reset_index().rename(columns={'tema_normalizado': 'tema'})

# Filtrar por similitud de títulos
medios_preferidos = ["CDMX","El Universal", "La Jornada", "Milenio", "Reforma", "Excélsior"]
df_grouped = filtrar_notas(df_grouped, umbral=75, medios_preferidos=medios_preferidos)

# Ordenar según prioridad de medios y alfabéticamente
df_grouped['orden_medio'] = df_grouped['medio'].apply(
    lambda x: medios_preferidos.index(x) if x in medios_preferidos else len(medios_preferidos)
)
df_grouped = df_grouped.sort_values(by=['orden_medio', 'medio', 'titulo']).drop(columns='orden_medio')
#Si el medio se llama "CDMX" cambiar por "Gobierno de la Ciudad de México"
df_grouped['medio'] = df_grouped['medio'].replace("CDMX", "Gobierno de la Ciudad de México")
#Cambiar "clara brugada" por "Jefa de Gobierno" y "gestion de riesgos" por "Gestión de Riesgos"
df_grouped['tema'] = df_grouped['tema'].replace({
    "Clara brugada": "Jefa de Gobierno",
    "Gestion de riesgos": "Gestión de Riesgos",
    "Espacio publico": "Espacio Público",
    "Participacion ciudadana": "Participación Ciudadana",
    "Planeacion": "Planeación",
    "Congreso de la ciudad de mexico": "Congreso de la Ciudad de México",
    "Transporte publico": "Transporte Público",
}, regex=True)

print(f'Total de notas después de agrupar y filtrar: {len(df_grouped)}')

Total de notas después de agrupar y filtrar: 118


In [55]:
df_grouped

,fecha_nota,medio,titulo,url,tema
31,2025-06-26,Gobierno de la Ciudad de México,Clara Brugada arranca Placatón contra la discr...,https://news.google.com/read/CBMi4gFBVV95cUxOR...,"Jefa de Gobierno, Espacio Público, Salud"
0,2025-06-25,Gobierno de la Ciudad de México,Firma DIF Ciudad de México Convenio de Colabor...,https://news.google.com/read/CBMi4gFBVV95cUxPS...,Salud
1,2025-06-25,Gobierno de la Ciudad de México,Se activa la alerta amarilla por lluvias fuert...,https://news.google.com/read/CBMi4wFBVV95cUxPW...,Gestión de Riesgos
44,2025-06-26,El Universal,Activan triple alerta por lluvias en toda la C...,https://news.google.com/read/CBMiyAFBVV95cUxOO...,Espacio Público
10,2025-06-25,El Universal,Congreso de CDMX pide a alcaldías limpiar dren...,https://news.google.com/read/CBMiigJBVV95cUxPc...,"Congreso de la Ciudad de México, Salud"
...,...,...,...,...,...
114,2025-06-26,TV Azteca Puebla,IMÁGENES FUERTES: Policías se autolesionan por...,https://news.google.com/read/CBMipwFBVV95cUxQT...,Seguridad ciudadana
115,2025-06-26,Telediario México,Calidad del aire CdMx: ¿hay contingencia ambie...,https://news.google.com/read/CBMihAFBVV95cUxNR...,"Medio ambiente, Participación Ciudadana, Salud..."
116,2025-06-26,Telediario México,Lluvias en CdMx y Edomex HOY 25 de junio: afec...,https://news.google.com/read/CBMinwFBVV95cUxPc...,"Agua, Gestión de Riesgos"
117,2025-06-26,Telediario México,¿Lloverá HOY 26 de junio en CdMx y Edomex? | P...,https://news.google.com/read/CBMilAFBVV95cUxQU...,Medio ambiente


In [56]:
#Crear un documento de Word
doc = Document()

In [57]:
#Homologar la fuente a Arial
style = doc.styles['Normal']
font = style.font
font.name = 'Arial'
font.size = Pt(10)

style.element.rPr.rFonts.set(qn('w:eastAsia'), 'Arial')

#Ponerlo en español utf-8
try:
    locale.setlocale(locale.LC_TIME, 'es_ES.UTF-8')
except:
    try:
        locale.setlocale(locale.LC_TIME, 'Spanish_Spain.1252')
    except:
        print('No se pudo establecer la localización en español. Usando traducción manual')

fecha_actual = pd.to_datetime('today')
try:
    fecha_formateada = fecha_actual.strftime('%d de %B de %Y')
except:
    fecha_formateada = fecha_en_espanol(fecha_actual)

In [58]:
#Formato de entrega
 
# Título
titulo = doc.add_heading("Monitoreo de Medios y Redes Sociales", level=1)
titulo.alignment = 1  

for run in titulo.runs:
    run.font.name = 'Arial'
    run.font.size = Pt(14)

# Subtítulo
subtitulo1 = doc.add_paragraph("Temas prioritarios de planeación de la Ciudad de México")
subtitulo1.alignment = 1  

for run in subtitulo1.runs:
    run.font.name = 'Arial'
    run.font.size = Pt(12)  

subtitulo2 = doc.add_paragraph(f"Informe con corte al día {fecha_formateada}")
subtitulo2.alignment = 1 

for run in subtitulo2.runs:
    run.font.name = 'Arial'
    run.font.size = Pt(12)  

#Crear tabla
table = doc.add_table(rows=1, cols=4)
table.style = 'Table Grid'

#Variables de la tabla
headers = ["Fecha de la nota", "Fuente", "Idea central", "Temas"]
for i, texto in enumerate(headers):
    p = table.rows[0].cells[i].paragraphs[0]
    run = p.add_run(texto)
    run.font.name = 'Arial'
    run.font.size = Pt(9)
    run._element.rPr.rFonts.set(qn('w:eastAsia'), 'Arial')

for _, row in df_grouped.iterrows():
    row_cells = table.add_row().cells

    #Fecha
    p_fecha = row_cells[0].paragraphs[0]
    run_fecha = p_fecha.add_run(row["fecha_nota"] if pd.notna(row["fecha_nota"]) else "No encontrado")
    run_fecha.font.name = 'Arial'
    run_fecha.font.size = Pt(9)
    run_fecha._element.rPr.rFonts.set(qn('w:eastAsia'), 'Arial')

    #Agregar hipervínculo
    p_medio = row_cells[1].paragraphs[0]
    add_hyperlink(p_medio, row["medio"], row["url"])

    #Título
    p_titulo = row_cells[2].paragraphs[0]
    run_titulo = p_titulo.add_run(row["titulo"])
    run_titulo.font.name = 'Arial'
    run_titulo.font.size = Pt(9)
    run_titulo._element.rPr.rFonts.set(qn('w:eastAsia'), 'Arial')

    #Tema
    p_tema = row_cells[3].paragraphs[0]
    run_tema = p_tema.add_run(row["tema"])
    run_tema.font.name = 'Arial'
    run_tema.font.size = Pt(9)
    run_tema._element.rPr.rFonts.set(qn('w:eastAsia'), 'Arial')

In [59]:
# Guardar
doc.save(f"Monitoreo_Medios_y_Redes_ {fecha_formateada}.docx")